In [ ]:
#| default_exp datasources.smp
%load_ext autoreload
%autoreload 2

import sys,os
from pathlib import Path

In [ ]:
# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))
os.chdir(Path.cwd().parent / 'extracao')

# Serviço Móvel Pessoal
> Módulo para encapsular a extração e processamento do Serviço Móvel Pessoal - Telefonia e Banda Larga Móvel - 2G, 3G, 4G e 5G

In [ ]:
#| export
import os
from functools import cached_property

import pandas as pd
import numpy as np
from dotenv import find_dotenv, load_dotenv

from extracao.constants import (
    AGG_SMP,
    CHANNELS,
    COLS_LICENCIAMENTO,
    DICT_LICENCIAMENTO,
    IBGE,
    MONGO_SMP,
    PROJECTION_LICENCIAMENTO,
)
from extracao.datasources.mosaico import Mosaico

In [ ]:
#| export
load_dotenv(find_dotenv())

True

In [ ]:
#| hide: true
#| eval:false
__file__ = Path.cwd().parent / 'extracao' / 'datasources.py'

In [ ]:
#| export

MONGO_URI = os.environ.get("MONGO_URI")

In [ ]:
#| export
class SMP(Mosaico):
    """Classe para encapsular a lógica de extração do SMP"""

    def __init__(self, mongo_uri: str = MONGO_URI, limit: int = 0) -> None:
        super().__init__(mongo_uri)
        self.limit = limit

    @property
    def stem(self):
        return "smp"

    @property
    def collection(self):
        return "licenciamento"

    @property
    def query(self):
        return MONGO_SMP

    @property
    def projection(self):
        return PROJECTION_LICENCIAMENTO

    @property
    def columns(self):
        return COLS_LICENCIAMENTO

    @property
    def cols_mapping(self):
        return DICT_LICENCIAMENTO

    @cached_property
    def extraction(self) -> pd.DataFrame:
        pipeline = [{"$match": self.query}, {"$project": self.projection}]
        if self.limit > 0:
            pipeline.append({"$limit": self.limit})
        df = self._extract(self.collection, pipeline)
        df["Log"] = ""
        return df

    def exclude_duplicated(
        self,
        df: pd.DataFrame,  # DataFrame com os dados de Estações
    ) -> pd.DataFrame:  # DataFrame com os dados duplicados excluídos
        f"""Exclui os registros duplicados
        O subconjunto de colunas consideradas é {AGG_SMP}
        """
        df["Número_Estação"] = df["Número_Estação"].astype("int")
        df = df.sort_values("Número_Estação", ignore_index=True)
        df["Largura_Emissão(kHz)"] = pd.to_numeric(
            df["Largura_Emissão(kHz)"], errors="coerce"
        )
        df["Largura_Emissão(kHz)"] = df["Largura_Emissão(kHz)"].fillna(0)
        df["Classe_Emissão"] = df["Classe_Emissão"].fillna("NI")
        df["Tecnologia"] = df["Tecnologia"].fillna("NI")
        duplicated = df.duplicated(subset=AGG_SMP, keep="first")
        df_sub = df[~duplicated].copy().reset_index(drop=True)
        discarded = df[duplicated].copy().reset_index(drop=True)
        log = f"""[("Colunas", {AGG_SMP}),  
                ("Processamento", "Registro agrupado e descartado do arquivo final")]"""
        self.append2discarded(self.register_log(discarded, log))
        for col in AGG_SMP:
            discarded_with_na = df_sub[df_sub[col].isna()]
            log = f"""[("Colunas", {col}),  
                    ("Processamento", "Registro com valor nulo presente")]"""
            self.append2discarded(self.register_log(discarded_with_na, log))
        df_sub = df_sub.dropna(subset=AGG_SMP)
        df_sub["Multiplicidade"] = df.groupby(AGG_SMP, sort=False).size().tolist()
        df_sub["Status"] = "L"
        df_sub["Fonte"] = "MOSAICO"
        log = f'[("Colunas", {AGG_SMP}), ("Processamento", "Agrupamento")]'
        return self.register_log(df_sub, log, df_sub["Multiplicidade"] > 1)

    @staticmethod
    def read_channels():
        channels = pd.read_csv(CHANNELS, dtype="string")
        cols = ["Downlink_Inicial", "Downlink_Final", "Uplink_Inicial", "Uplink_Final"]
        channels[cols] = channels[cols].astype("float")
        channels = channels.sort_values(["Downlink_Inicial"], ignore_index=True)
        channels["N_Bloco"] = channels["N_Bloco"].str.strip()
        channels["Faixa"] = channels["Faixa"].str.strip()
        return channels

    def exclude_invalid_channels(
        self,
        df: pd.DataFrame,  # DataFrame de Origem
    ) -> pd.DataFrame:  # DataFrame com os canais inválidos excluídos
        df_sub = df[df.Canalização == "Downlink"].copy().reset_index(drop=True)
        for flag in ["Uplink", "Inválida"]:
            discarded = df[df.Canalização == flag].copy()
            if not discarded.empty:
                log = f"""[("Colunas", ("Frequência", "Largura_Emissão(kHz)")),  
                         ("Processamento", "Canalização {flag}")]"""
                self.append2discarded(self.register_log(discarded, log))
        return df_sub

    def validate_channels(
        self,
        df: pd.DataFrame,  # DataFrame with the original channels info
    ) -> pd.DataFrame:  # DataFrame with the channels validated and added info
        """Read the SMP channels file, validate and merge the channels present in df"""
        bw = df["Largura_Emissão(kHz)"].astype("float") / 2000  # Unidade em kHz
        df["Início_Canal_Down"] = df.Frequência.astype(float) - bw
        df["Fim_Canal_Down"] = df.Frequência.astype(float) + bw
        channels = self.read_channels()
        grouped_channels = df.groupby(
            ["Início_Canal_Down", "Fim_Canal_Down"], as_index=False
        ).size()
        grouped_channels.sort_values(
            "size", ascending=False, inplace=True, ignore_index=True
        )
        grouped_channels["Canalização"] = "Inválida"
        grouped_channels.loc[:, "Offset"] = np.nan
        grouped_channels.loc[:, ["Blocos_Downlink", "Faixas"]] = pd.NA
        grouped_channels.loc[
            :, ["Blocos_Downlink", "Faixas", "Canalização"]
        ] = grouped_channels.loc[
            :, ["Blocos_Downlink", "Faixas", "Canalização"]
        ].astype(
            "string"
        )
        grouped_channels.loc[:, "Offset"] = grouped_channels.loc[:, "Offset"].astype(
            "float"
        )

        for row in grouped_channels.itertuples():
            interval = channels[
                (row.Início_Canal_Down < channels["Downlink_Final"])
                & (row.Fim_Canal_Down > channels["Downlink_Inicial"])
            ]
            faixa = "Downlink"
            if interval.empty:
                interval = channels[
                    (row.Início_Canal_Down < channels["Uplink_Final"])
                    & (row.Fim_Canal_Down > channels["Uplink_Inicial"])
                ]
                if interval.empty:
                    continue
                faixa = "Uplink"

            down = " | ".join(
                interval[["Downlink_Inicial", "Downlink_Final"]].apply(
                    lambda x: f"{x.iloc[0]}-{x.iloc[1]}", axis=1
                )
            )
            faixas = " | ".join(interval.Faixa.unique())
            if len(offset := interval.Offset.unique()) != 1:
                continue
            grouped_channels.loc[
                row.Index, ["Blocos_Downlink", "Faixas", "Canalização", "Offset"]
            ] = (down, faixas, faixa, float(offset[0]))
        grouped_channels = grouped_channels[
            [
                "Início_Canal_Down",
                "Fim_Canal_Down",
                "Blocos_Downlink",
                "Faixas",
                "Canalização",
                "Offset",
            ]
        ]
        df = pd.merge(
            df, grouped_channels, how="left", on=["Início_Canal_Down", "Fim_Canal_Down"]
        )
        return self.exclude_invalid_channels(df)

    def generate_uplink(
        self,
        df: pd.DataFrame,  # DataFrame de Origem
    ) -> pd.DataFrame:  # DataFrame com os canais de Uplink adicionados
        df["Offset"] = pd.to_numeric(df["Offset"], errors="coerce").astype("float")
        df["Largura_Emissão(kHz)"] = pd.to_numeric(
            df["Largura_Emissão(kHz)"], errors="coerce"
        ).astype("float")
        valid = (
            (df.Offset.notna())
            & (~np.isclose(df.Offset, 0))
            & (df["Largura_Emissão(kHz)"].notna())
            & (~np.isclose(df["Largura_Emissão(kHz)"], 0))
        )
        df.loc[:, ["Frequência", "Offset"]] = df.loc[
            :, ["Frequência", "Offset"]
        ].astype("float")
        df.loc[valid, "Frequência_Recepção"] = (
            df.loc[valid, "Frequência"] - df.loc[valid, "Offset"]
        )
        return df

    def substitute_coordenates(self, df: pd.DataFrame) -> pd.DataFrame:
        ibge = pd.read_csv(
            IBGE,
            dtype="string",
            usecols=["codigo_ibge", "nome", "latitude", "longitude"],
        )
        ibge.columns = ["Código_Município", "Município", "Latitude", "Longitude"]
        coords = pd.merge(
            df.loc[df.Multiplicidade > 1, "Código_Município"],
            ibge[["Código_Município", "Latitude", "Longitude"]],
            on="Código_Município",
            how="left",
        )
        df.loc[df.Multiplicidade > 1, ["Latitude", "Longitude"]] = coords[
            ["Latitude", "Longitude"]
        ].values
        log = """[("Colunas", ("Latitude", "Longitude")), 
        ("Processamento", "Substituição por Coordenadas do Município (Agrupamento)")]"""
        return self.register_log(df, log, df.Multiplicidade > 1)

    def input_fixed_columns(
        self,
        df: pd.DataFrame,  # DataFrame de Origem
    ) -> (
        pd.DataFrame
    ):  # DataFrame com os canais de downlink e uplink contenados e formatados
        df["Status"] = "L"
        df["Num_Serviço"] = "010"
        down = df.drop("Frequência_Recepção", axis=1)
        down["Fonte"] = "MOSAICO"
        down["Classe"] = "FB"
        up = df.drop("Frequência", axis=1)
        up = up.rename(columns={"Frequência_Recepção": "Frequência"})
        up.dropna(subset="Frequência", inplace=True)
        up["Fonte"] = "CANALIZACAO-SMP"
        up["Classe"] = "ML"
        return pd.concat([down, up], ignore_index=True)

    def _format(
        self,
        df: pd.DataFrame,  # DataFrame com os dados de Estações e Plano_Básico mesclados
    ) -> pd.DataFrame:  # DataFrame com os dados mesclados e limpos
        """Clean the merged dataframe with the data from the MOSAICO page"""
        df = df.rename(columns=self.cols_mapping)
        df = self.split_designacao(df)
        df = self.exclude_duplicated(df)
        df = self.validate_channels(df)
        df = self.generate_uplink(df)
        df = self.substitute_coordenates(df)
        df = self.input_fixed_columns(df)
        return df.loc[:, self.columns]

In [ ]:
#| export
if __name__ == "__main__":
    import time

    start = time.perf_counter()

    data = SMP()

    data.update()

    print("DATA")

    display(data.df)

    print(150 * "=")

    print("DISCARDED!")

    display(data.discarded[["Frequência", "Entidade", "Log"]])

    print(150 * "=")

    print(data.df.Multiplicidade.sum())

    data.save()

    print(f"Elapsed time: {time.perf_counter() - start} seconds")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/melinda/micromamba/envs/dados/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_1457/2230402201.py", line 9, in <module>
    data.update()
  File "/mnt/c/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/base.py", line 117, in update
    self.df = self._format(self.extraction)
  File "/home/melinda/micromamba/envs/dados/lib/python3.8/functools.py", line 967, in __get__
    val = self.func(instance)
  File "/tmp/ipykernel_1457/2279196232.py", line 38, in extraction
    df = self._extract(self.collection, pipeline)
  File "/mnt/c/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/mosaico.py", line 49, in _extract
    result = collection.aggregate(pipeline)
  File "/home/melinda/micromamba/envs/dados/lib/python3.8/site-packages/pymongo/collection.py", line 2721, in aggregate
    with self.__database

In [ ]:
# import nbdev; nbdev.nbdev_export()